In [106]:
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import sklearn
import sys
import tensorflow as tf
import time
from PIL import Image
import cv2
from scipy import signal
from keras import backend as K
from tensorflow import keras
from tensorflow.keras.preprocessing import image
print(tf.__version__)

2.1.0


### im2col方法

In [163]:
def im2col(input_data, filter_h, filter_w, stride=1, pad=0):
    """

    Parameters
    ----------
    input_data : 由(数据量, 通道, 高, 长)的4维数组构成的输入数据
    filter_h : 滤波器的高
    filter_w : 滤波器的长
    stride : 步幅
    pad : 填充

    Returns
    -------
    col : 2维数组
    """
    N, C, H, W = input_data.shape
    out_h = (H + 2*pad - filter_h)//stride + 1
    out_w = (W + 2*pad - filter_w)//stride + 1

    img = np.pad(input_data, [(0,0), (0,0), (pad, pad), (pad, pad)], 'constant')
    col = np.zeros((N, C, filter_h, filter_w, out_h, out_w))

    for y in range(filter_h):
        y_max = y + stride*out_h
        for x in range(filter_w):
            x_max = x + stride*out_w
            col[:, :, y, x, :, :] = img[:, :, y:y_max:stride, x:x_max:stride]

    col = col.transpose(0, 4, 5, 1, 2, 3).reshape(N*out_h*out_w, -1)
    return col

In [164]:
sample = np.arange(16).astype("float32").reshape([1,1,4,4])

In [165]:
input = im2col(sample, 3, 3, stride=1, pad=0)

In [166]:
input

array([[ 0.,  1.,  2.,  4.,  5.,  6.,  8.,  9., 10.],
       [ 1.,  2.,  3.,  5.,  6.,  7.,  9., 10., 11.],
       [ 4.,  5.,  6.,  8.,  9., 10., 12., 13., 14.],
       [ 5.,  6.,  7.,  9., 10., 11., 13., 14., 15.]])

In [167]:
kernal1 = np.array([1,-1,1,0,-1,0,0,1,0]).astype("float32").reshape(3,3)

In [168]:
kernal = kernal1.reshape(-1,1)

In [169]:
kernal.shape

(9, 1)

In [170]:
out = np.dot(input,kernal)

In [172]:
out.reshape([2,2])

array([[ 5.,  6.],
       [ 9., 10.]])

In [173]:
sample[0,0,:,:]

array([[ 0.,  1.,  2.,  3.],
       [ 4.,  5.,  6.,  7.],
       [ 8.,  9., 10., 11.],
       [12., 13., 14., 15.]], dtype=float32)

In [183]:
y = signal.correlate2d(sample[0,0,:,:], kernal1, mode='valid')

In [184]:
y

array([[ 5.,  6.],
       [ 9., 10.]], dtype=float32)

### unroll kernal

In [62]:
input_np = np.arange(16).astype("float32").reshape(-1,1)

In [63]:
input_np

array([[ 0.],
       [ 1.],
       [ 2.],
       [ 3.],
       [ 4.],
       [ 5.],
       [ 6.],
       [ 7.],
       [ 8.],
       [ 9.],
       [10.],
       [11.],
       [12.],
       [13.],
       [14.],
       [15.]], dtype=float32)

In [64]:
kernal_np = np.array([1,-1,1,0,-1,0,0,1,0]).astype("float32").reshape(3,3)

In [65]:
kernal_np 

array([[ 1., -1.,  1.],
       [ 0., -1.,  0.],
       [ 0.,  1.,  0.]], dtype=float32)

In [66]:
def unroll_kernel(kernel, n):

    m = kernel.shape[0]
    unrolled_K = np.zeros(((n - m + 1)**2, n**2))

    skipped = 0
    for i in range(n ** 2):
         if (i % n) < n - m + 1 and((i / n) % n) < n - m + 1:
             for j in range(m):
                 for l in range(m):
                    unrolled_K[i - skipped, i + j * n + l] = kernel[j, l]
         else:
             skipped += 1
    return unrolled_K

In [67]:
unroll_k = unroll_kernel(kernal_np,4)
unroll_k

array([[ 1., -1.,  1.,  0.,  0., -1.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,
         0.,  0.,  0.],
       [ 0.,  1., -1.,  1.,  0.,  0., -1.,  0.,  0.,  0.,  1.,  0.,  0.,
         0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  1., -1.,  1.,  0.,  0., -1.,  0.,  0.,  0.,
         1.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  1., -1.,  1.,  0.,  0., -1.,  0.,  0.,
         0.,  1.,  0.]])

In [68]:
result = np.dot(unroll_k,input_np)
result

array([[ 5.],
       [ 6.],
       [ 9.],
       [10.]])

In [69]:
np.dot(unroll_k.T,result).reshape([4,4])

array([[ 5.,  1., -1.,  6.],
       [ 9., -4., -7., 10.],
       [ 0., -4., -4.,  0.],
       [ 0.,  9., 10.,  0.]])

### tf.nn.conv2d

In [70]:
input_tf = tf.constant(input_np,shape=[1,4,4,1])
input_tf[0,:,:,0]

<tf.Tensor: shape=(4, 4), dtype=float32, numpy=
array([[ 0.,  1.,  2.,  3.],
       [ 4.,  5.,  6.,  7.],
       [ 8.,  9., 10., 11.],
       [12., 13., 14., 15.]], dtype=float32)>

In [71]:
kernal_tf = tf.constant(kernal_np,shape=[3,3,1,1])

In [72]:
value = tf.nn.conv2d(input_tf, kernal_tf, strides=[1, 1, 1, 1], padding='VALID')

In [73]:
value

<tf.Tensor: shape=(1, 2, 2, 1), dtype=float32, numpy=
array([[[[ 5.],
         [ 6.]],

        [[ 9.],
         [10.]]]], dtype=float32)>

In [74]:
result = tf.nn.conv2d_transpose(value, kernal_tf, output_shape=[1,4,4,1], strides=[1, 1, 1, 1], padding='VALID')

In [75]:
result[0,:,:,0].numpy

<bound method _EagerTensorBase.numpy of <tf.Tensor: shape=(4, 4), dtype=float32, numpy=
array([[ 5.,  1., -1.,  6.],
       [ 9., -4., -7., 10.],
       [ 0., -4., -4.,  0.],
       [ 0.,  9., 10.,  0.]], dtype=float32)>>

### tf.keras.layers.Conv2D

In [76]:
def kernel_init(shape, dtype=None):
    b = K.constant([[[ 1., -1.,  1.],
       [ 0., -1.,  0.],
       [ 0.,  1.,  0.]]],shape=[3,3,1,1])
    return b

In [77]:
input_tf

<tf.Tensor: shape=(1, 4, 4, 1), dtype=float32, numpy=
array([[[[ 0.],
         [ 1.],
         [ 2.],
         [ 3.]],

        [[ 4.],
         [ 5.],
         [ 6.],
         [ 7.]],

        [[ 8.],
         [ 9.],
         [10.],
         [11.]],

        [[12.],
         [13.],
         [14.],
         [15.]]]], dtype=float32)>

In [177]:
conv = tf.keras.layers.Conv2D(filters=1,kernel_size=3,strides=(1,1),padding='valid',kernel_initializer=kernel_init)(input_tf)
conv

<tf.Tensor: shape=(1, 2, 2, 1), dtype=float32, numpy=
array([[[[ 5.],
         [ 6.]],

        [[ 9.],
         [10.]]]], dtype=float32)>

In [175]:
result = tf.keras.layers.Conv2DTranspose(filters=1,kernel_size=3,strides=(1,1),padding='valid',kernel_initializer=kernel_init)(conv)

In [176]:
result[0,:,:,0].numpy

<bound method _EagerTensorBase.numpy of <tf.Tensor: shape=(4, 4), dtype=float32, numpy=
array([[ 5.,  1., -1.,  6.],
       [ 9., -4., -7., 10.],
       [ 0., -4., -4.,  0.],
       [ 0.,  9., 10.,  0.]], dtype=float32)>>